# Spotify Podcast Dataset - Publisher



In [11]:
import pandas as pd
from scipy.cluster.hierarchy import linkage, dendrogram
import matplotlib.pyplot as plt

In [12]:
df = pd.read_csv('../../podcasts-no-audio-13GB-selected/metadata.tsv', sep='\t')

## Find Shows by Publisher

In [13]:
df_pub2show = df.groupby(['publisher','show_name']).size().sort_values(ascending=False).reset_index().groupby('publisher').size().sort_values(ascending=False)
df_pub2show = pd.DataFrame(df_pub2show, columns=['Number of Shows']).reset_index()
# Calculate circle sizes based on 'Value'
#df_pub2show['CircleSize'] = np.sqrt(df_pub2show['Number of Shows'])
df_pub2show['Number of Shows Jitter'] = df_pub2show['Number of Shows'] + np.abs(np.random.normal(0,1,(len(df_pub2show),)))

## Find Episodes by Publisher

In [14]:
df_pub2episode = df.groupby('publisher').size().sort_values(ascending=False)
df_pub2episode = pd.DataFrame(df_pub2episode, columns=['Number of Episodes']).reset_index()
# Calculate circle sizes based on 'Value'
df_pub2episode['CircleSize'] = np.sqrt(df_pub2episode['Number of Episodes'])
df_pub2episode['Number of Episodes Jitter'] = df_pub2episode['Number of Episodes'] + np.random.random((len(df_pub2episode),))
df_pub2episode.shape

(17490, 4)

In [15]:
df_pub = df_pub2episode.merge(df_pub2show,on='publisher',how='left')

In [16]:
import altair as alt
import pandas as pd
import numpy as np

# Create a scatter plot with circles to represent the categorical table
circle_table = alt.Chart(df_pub.sample(5000)).mark_circle().encode(
    x=alt.X('publisher:O', axis=alt.Axis(labels=False, title='')),
    y=alt.Y('Number of Episodes Jitter:Q', scale=alt.Scale(type='log'), axis=alt.Axis(labels=False, title='')),
    size=alt.Size('CircleSize:Q', scale=alt.Scale(range=[5, 2000]), legend=None),  # Adjust size range as needed
    #color=alt.Color('publisher:N', legend=None),
    color=alt.Color(scale=alt.Scale(scheme='blueorange-8')),
    tooltip=['publisher:N', 'Number of Episodes:Q','Number of Shows:Q']  # Add a tooltip with 'Category' and 'Value'
).properties(
    width=700,
    height=700,
    title='Podcast Publishers (Hover Over)'
).configure_title(
    fontSize=16,
    anchor='middle'
).configure_axis(
    grid=False  # Remove gridlines
).configure_axisY(
    ticks=False
)

circle_table

alt.Chart(...)

In [17]:
df.columns

Index(['show_uri', 'show_name', 'show_description', 'publisher', 'language',
       'rss_link', 'episode_uri', 'episode_name', 'episode_description',
       'duration', 'show_filename_prefix', 'episode_filename_prefix'],
      dtype='object')

In [18]:
min(np.random.normal(0,1,(len(df_pub2show),)))

-3.9612177097080754

In [19]:
df[df['publisher']=='David'].groupby('show_filename_prefix').size()

show_filename_prefix
show_0tdzFF8Qse4UR8pn9ReNL1    1
show_11UVF4OT5CAlvb5sZCsXu0    2
show_1avz49Hd2W7nj2XXb0SQu1    1
show_2b9g1RKvE30BKJDkkui16u    5
show_6V4blubY5N1PfwpZjKMoCI    3
show_7ndWuoDaVWUbdMdtGCr6ab    1
dtype: int64